# Setup

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from torchmetrics import Accuracy

from sklearn.metrics import classification_report

In [2]:
from IPython.display import clear_output
import os, sys, shutil
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm, trange
from glob import glob
import random

import cv2 as cv

In [3]:
print(os.getcwd())

D:\GitCloneProject\HeartResearch\Experiment\Approach\EfficientB0


In [4]:
for i in range(3):
    os.chdir("..")

print(os.getcwd())

D:\GitCloneProject\HeartResearch


In [5]:
# D:\GitCloneProject\HeartResearch\Data set\v4_data\med_scaleogram_h256_w512_seglen1600_scl500
main_data_dir = os.getcwd() + "\\Data set"

label_csv_path = main_data_dir + "\\Label.csv"

img_data_dir = main_data_dir + "\\v4_data\\med_scaleogram_h256_w512_seglen1600_scl500"
print(len(os.listdir(img_data_dir)))

425820


In [6]:
label_df = pd.read_csv(label_csv_path)
label_df.head()

,Recording,First_label,Second_label,Third_label
0,A0001,5,NaN,NaN
1,A0002,1,NaN,NaN
2,A0003,2,NaN,NaN
3,A0004,2,NaN,NaN
4,A0005,7,NaN,NaN


In [7]:
label_df["First_label"].value_counts()

5    1695
2    1098
1     918
8     826
3     704
7     653
6     574
4     207
9     202
Name: First_label, dtype: int64

 # Data Loader

In [8]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.nn.functional import one_hot

In [ ]:
class HeartData():
    def __init__(self):
        pass
    
    def __getdata__(self):
        
        pass
        

In [9]:
class HeartDisease(Dataset):
    def __init__(self, label_df: pd.DataFrame = label_df, 
                 root_dir: str = img_data_dir, 
                 ratio: tuple = (0.8, 0.1, 0.1),
                 subset = "training", # validating, testing
                 resize: tuple = None, #  (256, 512)
                 seed: int = 777
                ): 
        
        self.label_df = label_df
        self.label_col = torch.tensor(self.label_df["First_label"].values - np.ones_like(self.label_df["First_label"].values.shape))
        self.onehot_label = one_hot(self.label_col)
        
        self.label_dict = {
            name : vector for name, vector in zip(self.label_df["Recording"], self.onehot_label)
        }
        
        self.root_dir = root_dir
        self.ratio = ratio
        self.subset = subset
        self.resize = resize
        self.fullpaths = glob(self.root_dir + "\\*")    
        random.Random(seed).shuffle(self.fullpaths)
        self.start = int(len(self.fullpaths) * self.ratio[0])
        self.mid = int(len(self.fullpaths) * self.ratio[1])
        self.stop = int(len(self.fullpaths) * self.ratio[2])
        if self.subset == "training":
            self.imgpaths = self.fullpaths[ : self.start]
        elif self.subset == "validating":
            self.imgpaths = self.fullpaths[self.start : self.start + self.mid]
        elif self.subset == "testing":
            self.imgpaths = self.fullpaths[self.start + self.mid : self.start + self.mid + self.stop]
        else:
            raise Exception(f'Invalid subset. Subset Arg should be training, validating, and testing but "{self.subset}" found')

    def __len__(self):
        return len(self.imgpaths)

    def __getitem__(self, idx:list = None):
        if idx == None:
            raise Exception('idx arg cannot be None')
            
        imgs = []
        labels= []
        
        for i in idx:
            temp_path = self.imgpaths[i]
            
            filename = temp_path.split("\\")[-1].split("_")[0]
            labels.append(self.label_dict[filename])
            
            
            temp_img = cv.imread(temp_path)
            temp_img = torch.tensor(temp_img).permute(-1, 0, 1)
            imgs.append(temp_img)
        
        batch_imgs = torch.stack(imgs)
        batch_labels = torch.stack(labels)

        return batch_imgs, batch_labels

# Model 

In [2]:
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch import nn

ori_model = efficientnet_b0(weights = EfficientNet_B0_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /Users/hoangminhthao/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)>

In [ ]:
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch import nn

class HeartModel(): #tạo model từ EfficientNet_B0
    def __init__(self):
        

In [11]:
model

HeartModel(
  (ori_model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scale_

# Training

In [12]:
from datetime import datetime

opt_mapping = {
    "Adam" : torch.optim.Adam
}

loss_mapping = {
    "CCE" : nn.CrossEntropyLoss()
}

class Training:
    def __init__(self, 
                 device: str = "cuda",
                 learning_rate:float = 0.0001,
                 optimizer:str = "Adam",
                 loss:str = "CCE",
                 model = model,
                 batchsize:int = 32,
                 epochs:int = 1,
                 label_df: pd.DataFrame = label_df, 
                 root_dir: str = img_data_dir, 
                 ratio: tuple = (0.8, 0.1, 0.1),
                 mode:str = "development",
                 resize: tuple = None, #  (256, 512)
                 seed: int = 777,
                 early_stop: dict = {
                     "patience" : 10,
                     "object" : "train_acc", # train_loss, val_loss, val_acc
                 },
                 
                 model_check_point: bool = True
                ):
        
        # Setup
        self.device = device
        self.current_time = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
        self.model = model
        self.model.to(self.device)
        self.lr = learning_rate
        self.optimizer = opt_mapping[optimizer](self.model.parameters(), lr=self.lr)
        self.loss_fn = loss_mapping[loss]
        self.bs = batchsize
        self.ep = epochs
        self.default_ratio = (0.001, 0.001, 0.001)
        self.accuracy = Accuracy(task="multiclass", num_classes=9)
        self.target_names = [str(i) for i in range(9)]
        
        if early_stop:
            self.early_stop = early_stop
        
        # Data
        self.label_df = label_df
        self.root_dir = root_dir
        if mode == "experiment":
            self.ratio = ratio
        elif mode == "development":
            self.ratio = self.default_ratio
        self.resize = resize
        self.seed = seed
        
        self.train_data = HeartDisease(label_df = self.label_df, 
                 root_dir = self.root_dir, 
                 ratio = self.ratio,
                 subset = "training",
                 resize = self.resize,
                 seed = self.seed)
        
        self.valid_data = HeartDisease(label_df = self.label_df, 
                 root_dir = self.root_dir, 
                 ratio = self.ratio,
                 subset = "validating",
                 resize = self.resize,
                 seed = self.seed)
        
        self.test_data = HeartDisease(label_df = self.label_df, 
                 root_dir = self.root_dir, 
                 ratio = self.ratio,
                 subset = "testing",
                 resize = self.resize,
                 seed = self.seed)
        
        # Logging
        self.log_dict = {
            "train_loss" : [],
            "train_acc" : [],
            "val_loss" : [],
            "val_acc" : []
        }
    
    def get_sample_count(self):
        print(f"Training: {len(self.train_data)}")
        print(f"Validating: {len(self.valid_data)}")
        print(f"Testing: {len(self.test_data)}")
    
    def logging(self, train_loss:float , 
                train_acc: float, valid_loss:float, valid_acc:float):
        self.log_dict["train_loss"].append(train_loss)
        self.log_dict["train_acc"].append(train_acc)
        self.log_dict["valid_loss"].append(valid_loss)
        self.log_dict["valid_acc"].append(valid_acc)
    
    def export_log(self, rdir:str = None, extenstion:str = "parquet"):
        if not rdir:
            file_path = os.getcwd() + f"/result/"
        pass
    
    def export_model(self): # filename = val_loss + val_acc
        pass
    
    def clear_buffer(self):
        del self.train_data
        del self.test_data
        del self.valid_data
        del self.model
    
    def update(self):
        if early_stop:
            self.checkpoint = 0
            if "acc" in early_stop["object"]:
                self.last_value = 0
            elif "loss" in early_stop["object"]:
                self.last_value = 100
        
        self.model.train()
        for e in range(self.ep):
            for batch in trange(0, len(self.train_data), self.bs):
                if len(self.train_data) - batch > self.bs:
                    batch_indices = [x for x in range(batch, batch + self.bs)]
                else:
                    batch_indices = [x for x in range(batch, len(self.train_data))]
                X, y = self.train_data[batch_indices]

                pred = self.model((X/255).to(self.device))
                loss = self.loss_fn(pred, y.to(self.device, dtype = torch.float))

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                
            train_acc = self.accuracy(torch.argmax(pred, dim = 1), torch.argmax(y, dim = 1))

            val_loss, val_acc = self.validation()

            # Show train_loss, train_acc, val_loss, val_acc
            print(f"Epoch: {e} - Train Loss: {loss.item()} - Train Acc: {train_acc} - Val Loss: {val_loss.item()} - Val Acc: {val_acc.item()}")
            
            if early_stop:
                if early_stop["object"] == "train_loss":
                    if loss.item() > self.last_value:
                        self.check_point += 1
                    else:
                        self.last_value = loss.item()
                elif early_stop["object"] == "train_acc":
                    if train_acc < self.last_value:
                        self.check_point += 1
                    else:
                        self.last_value = train_acc
                elif early_stop["object"] == "val_loss":
                    if val_loss.item() > self.last_value:
                        self.check_point += 1
                    else:
                        self.last_value = val_loss.item()
                elif early_stop["object"] == "val_acc":
                    if val_acc < self.last_value:
                        self.check_point += 1
                    else:
                        self.last_value = val_acc
                else:
                    raise Exception(f"early_stop arg should have the value val_acc, val_loss, train_acc, train_loss but found {early_stop["object"]} instead")
            
            elif model_check_point:
                
                

        if self.check_point >= self.early_stop["patience"]:
            break
            
    
    def validation(self):
        loss = 0
        target = []
        preds = []
        
        with torch.no_grad():
            for batch in trange(0, len(self.valid_data), self.bs):
                if len(self.valid_data) - batch > self.bs:
                    batch_indices = [x for x in range(batch, batch + self.bs)]
                else:
                    batch_indices = [x for x in range(batch, len(self.valid_data))]
                X, y = self.valid_data[batch_indices]
                pred = self.model((X/255).to(self.device))

                loss_out = self.loss_fn(pred, y.to(self.device, dtype = torch.float))
                
                loss += loss_out
                
                target.append(y[0])
                preds.append(pred[0])
                
            target = torch.stack(target)
            preds = torch.stack(preds)
            loss = loss/len(self.valid_data)
            
            return loss, self.accuracy(torch.argmax(preds, dim = 1), torch.argmax(target, dim = 1))
        
    def evaluation(self):
        target = []
        preds = []
        
        with torch.no_grad():
            for batch in range(len(self.test_data)):
                X, y = self.test_data[[batch]]
                pred = self.model((X/255).to(self.device))
                
                target.append(y[0])
                preds.append(pred[0])
                
            target = torch.stack(target)
            preds = torch.stack(preds)
            
            # Classification report
            print(classification_report(
                torch.argmax(target, dim=1).numpy().tolist(), 
                torch.argmax(preds, dim=1).numpy().tolist(), 
                target_names=self.target_names))            
            
            # Confusion Matrix
            
            # ROC Curve
            

SyntaxError: f-string: unmatched '[' (4287155738.py, line 169)

In [ ]:
# Metadata

# Training Setup
device = "cpu"
learning_rate = 0.0001
optimizer = "Adam"
loss = "CCE"
batchsize = 128
epochs = 50
ratio = (0.8, 0.1, 0.1)
mode = "development"
seed = 123

# Data

monitor = Training(device = device,
                   learning_rate = learning_rate,
                   optimizer = optimizer,
                   loss = loss,
                   model = model,
                   batchsize = batchsize,
                   epochs = epochs,
                   label_df = label_df, 
                   root_dir = img_data_dir, 
                   ratio = ratio,
                   mode = mode,
                   seed = seed)

In [ ]:
# Unitest

monitor.get_sample_count()

In [ ]:
monitor.update()

# Evaluation

In [ ]:
monitor.evaluation()

# Clear Buffer

In [ ]:
# monitor.clear_buffer()